# Dependences

In [1]:
using CUDA
using DelimitedFiles
using Test
using BenchmarkTools

# initial data
R_Agg = 13
r_max = 3.0
n_knn = 50
idx_red_size =  r_max ≤ 2.80 ? 13 :
            2.80 < r_max ≤ 3.45 ? 21 :
            3.45 < r_max ≤ 3.80 ? 39 :
            3.80 < r_max ≤ 4.00 ? 55 :
            70
X = Float64.(readdlm("../../../data/init/Sphere/$(R_Agg).xyz")[3:end,2:end]) |> cu

# kNN
idx      = Int32.(zeros(size(X, 1), size(X, 1))) |> cu;
idx_sum  = Int32.(zeros(1, size(idx, 1))) |> cu
idx_red  = Int32.(zeros(idx_red_size, size(idx, 1))) |> cu
idx_contractile =Int32.(zeros(n_knn,size(X,1))) |> cu

# forces
force = CUDA.zeros(size(X));

# Generic Functions

In [2]:
# force_func
cubic(A,r_max,r_min,dist) = - A * (dist-r_max)^2* (dist-r_min)

# distances
euclidean(points,i,j) = sqrt((points[i,1]-points[j,1])^2+(points[i,2]-points[j,2])^2+(points[i,3]-points[j,3])^2)

euclidean (generic function with 1 method)

# idx

## functions

In [3]:
function dist_kernel!(idx, points ,r_max)
    # Defining Index for kernel
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    j = (blockIdx().y - 1) * blockDim().y + threadIdx().y
    
    # Limiting data inside matrix
    if i <= size(points, 1) && j <= size(points, 1)
        if euclidean(points,i,j) < r_max
            idx[i, j] = i
        else
            idx[i, j] = 0
        end 
    end
    return nothing
end

function reduce_kernel(idx,idx_red,idx_sum)
    # Defining Index for kernel
    i  = (blockIdx().x-1) * blockDim().x + threadIdx().x

    # Limiting data inside matrix
    if i <= size(idx,1)
        # Cleaning idx_sum
        idx_sum[i] = 0
        
        # looping on each row for searching non-zero values
        for j = 1:size(idx,1)
            if idx[j,i] != 0
                idx_sum[i] += 1
                idx_red[idx_sum[i],i] = j
            end
        end
    end
    
    return nothing
end

function index_contractile!(idx_contractile,idx_sum,idx_red)
    # Defining Index for kernel
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    j = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    # Limiting data inside matrix
    if i <= size(idx_contractile, 1) && j <= size(idx_contractile,2)
        idx_contractile[i,j] = idx_red[rand(1:idx_sum[j]),j]
    end
    return nothing
end

function nearest_neighbors(idx, idx_red, idx_sum, idx_contractile, points ,r_max)
    # Calculating Distance Matrix
    threads =(32,32)
    blocks  =cld.(size(points,1),threads)
    @cuda threads=threads blocks=blocks dist_kernel!(idx, points ,r_max)

    # Reducing Distance Matrix to Nearest Neighbors
    threads=1024
    blocks=cld.(size(idx,1),threads)
    @cuda threads=threads blocks=blocks reduce_kernel(idx,idx_red,idx_sum)

    # Finding index contractile
    threads =(32,32)
    blocks  =cld.(size(X,1),threads)
    @cuda threads=threads blocks=blocks index_contractile!(idx_contractile,idx_sum,idx_red)
end

nearest_neighbors (generic function with 1 method)

## running

In [4]:
println("---------------------------------- VARIABLES ------------------------------------")
println("R_Agg = $(R_Agg) | R_Max = $(r_max) | col_size_idx = $(idx_red_size)")
display(X)
println("------------------------------------ SIZES --------------------------------------")
println("Size → idx     = $(size(idx))")
println("     → idx_sum = $(size(idx_sum))")
println("     → idx_red = $(size(idx_red))")
println("------------------------------------ RESULTS ------------------------------------")
@btime nearest_neighbors(idx, idx_red, idx_sum,idx_contractile, X ,r_max)
println("----------------------------------- IDX RED -------------------------------------")
display(idx_red)
println("----------------------------------- IDX SUM -------------------------------------")
display(idx_sum)
println("------------------------------- IDX CONTRACTILE ---------------------------------")
display(idx_contractile)
println("---------------------------------- Memory Used ----------------------------------")
CUDA.memory_status()  

---------------------------------- VARIABLES ------------------------------------
R_Agg = 13 | R_Max = 3.0 | col_size_idx = 21


1620×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.5  -4.04  -12.25
  1.5  -4.04  -12.25
 -3.5  -2.31  -12.25
 -1.5  -2.31  -12.25
  0.5  -2.31  -12.25
  2.5  -2.31  -12.25
 -2.5  -0.58  -12.25
 -0.5  -0.58  -12.25
  1.5  -0.58  -12.25
  3.5  -0.58  -12.25
 -3.5   1.15  -12.25
 -1.5   1.15  -12.25
  0.5   1.15  -12.25
  ⋮           
  0.5  -1.15   12.25
  2.5  -1.15   12.25
 -2.5   0.58   12.25
 -0.5   0.58   12.25
  1.5   0.58   12.25
  3.5   0.58   12.25
 -3.5   2.31   12.25
 -1.5   2.31   12.25
  0.5   2.31   12.25
  2.5   2.31   12.25
 -0.5   4.04   12.25
  1.5   4.04   12.25

------------------------------------ SIZES --------------------------------------
Size → idx     = (1620, 1620)
     → idx_sum = (1, 1620)
     → idx_red = (21, 1620)
------------------------------------ RESULTS ------------------------------------
  12.990 μs (77 allocations: 4.03 KiB)
----------------------------------- IDX RED -------------------------------------


21×1620 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
  1   1   3   1   1   2   3   4   5  …  1580  1581  1582  1583  1589  1590
  2   2   4   3   2   5   4   5   6     1581  1582  1583  1584  1590  1591
  4   5   7   4   4   6   7   7   8     1582  1583  1584  1585  1591  1592
  5   6  23   5   5   9   8   8   9     1588  1589  1590  1591  1596  1597
 19  20  29   7   6  10  11   9  10     1589  1590  1591  1592  1597  1598
 24  25  30   8   8  26  12  12  13  …  1595  1596  1597  1598  1601  1602
 25  26  35  24   9  32  30  13  14     1611  1611  1612  1613  1616  1617
 30  31  36  30  25  33  36  31  32     1615  1612  1613  1614  1617  1618
 31  32  37  31  31  38  37  37  38     1616  1615  1616  1617  1619  1619
 32  33   0  36  32  39  43  38  39        0  1616  1617  1618  1620  1620
  0   0   0  37  37  40  44  44  45  …     0  1617  1618  1620     0     0
  0   0   0  38  38   0  45  45  46        0  1619  1619     0     0     0
  0   0   0   0  39   0   0  46  47        0     0

----------------------------------- IDX SUM -------------------------------------


1×1620 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 10  10  9  12  13  11  12  13  13  10  …  13  13  10  9  12  13  11  10  10

------------------------------- IDX CONTRACTILE ---------------------------------


50×1620 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 30  31   3   1  39  10   7  38  47  …  1582  1582  1582  1620  1617  1590
 30  32  23  24   9   2   3  46  46     1580  1583  1618  1598  1589  1619
 25  32  29   1  32   5  30  38   5     1580  1611  1619  1613  1596  1598
 32  32  36  30  38   5  30   9  32     1581  1583  1597  1598  1596  1592
  4   2   3   3  37  10  44  38  38     1611  1589  1616  1614  1596  1592
 30  32  35   4   8  40  36   5  45  …  1582  1590  1590  1584  1596  1591
  1  26  37   8   4  39   8  13   6     1611  1619  1617  1585  1620  1602
 19  31  36   7   8  38  37   7  47     1595  1619  1617  1617  1601  1592
 30  20   3  37   1  10  11   9  45     1588  1582  1591  1585  1617  1591
 19   6  35  38  32   9   4   8  39     1581  1611  1618  1598  1597  1597
 31   1  37  37   5  26  36  13  10  …  1588  1581  1583  1617  1596  1618
 19   2  37   5  31   2   3   4  46     1616  1617  1612  1585  1596  1602
 31  20  29   8  25  40  11   9  32     1580  1581

---------------------------------- Memory Used ----------------------------------
Effective GPU memory usage: 30.49% (610.562 MiB/1.955 GiB)
No memory pool is in use.

# forces

## functions

In [5]:
function sum_force!(idx,idx_contractile,points,force, force_func,A,r_max,r_min,A_con,t_knn)
    # Defining Index for kernel
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    k = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    # Limiting data inside matrix
    if i <= size(points, 1) && k <= size(points, 2)

        # Cleaning idx_sum
        force[i,k] = 0
        dist = 0

        # Iterate on each row
        for j=1:size(idx,1)

            # Finding forces
            if idx[j,i] != i && idx[j,i] != 0
                dist = euclidean(points,i,idx[j,i])
                force[i,k] +=force_func(A,r_max,r_min,dist) * (points[i,k]-points[idx[j,i],k]) / dist
            end
            
        end

        # Adding Contrractile Force
        if idx_contractile[t_knn,i] != i && idx_contractile[t_knn,i] != 0
            dist = euclidean(points,i,idx_contractile[t_knn,i])
            force[i,k] += A_con*(points[i,k]-points[idx_contractile[t_knn,i],k])/dist
        end
        
    end
    return nothing
end

sum_force! (generic function with 1 method)

## running

In [6]:
dt = 0.5
A     = 0.1
A_con = 0.001
r_min = 2.0
r_max = r_max

t_knn = 1

1

In [13]:
# GPU
threads =(100,3)
blocks  =cld.(size(X,1),threads)
@btime @cuda threads=threads blocks=blocks sum_force!(idx_red,idx_contractile, X ,force, cubic,A,r_max,r_min,A_con,Int(t_knn))
display(force)

  7.146 μs (52 allocations: 2.64 KiB)


1620×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.000706573  -0.000870194   0.00482211
  0.000706573  -0.000870194   0.00482211
 -8.91818f-5   -5.35724f-5    0.00573757
 -0.000589626   0.000812196   0.00573757
 -0.000705556  -0.00030851    0.00515901
 -0.000411262  -0.000919341   0.00573757
 -2.89773f-11   0.000266457   0.00527907
 -0.000499096   0.000550942   0.00446055
 -0.000705556  -0.000142765   0.00470051
  0.000178364   0.00108186    0.00470017
 -0.000251113   0.00170953    0.00540328
 -2.89773f-11   0.00240127    0.00482271
  0.000705556   0.00117784    0.00482472
  ⋮                          
 -3.21325f-11  -0.0010096    -0.00458687
 -0.00083761   -0.00128253   -0.00458476
 -2.48566f-11   0.000311007  -0.00446266
  0.000500444   0.000599311  -0.00527907
 -0.000500444   0.000599311  -0.00527907
  0.00117836   -0.000266457  -0.00527907
 -0.000794738   0.000462795  -0.00515901
 -8.91819f-5    0.000631037  -0.00492116
  0.000705556   0.00030851   -0.00515901
  0.000589626  -0

## Pending (Test with CPU)

In [17]:
# CPU
X_CPU = Matrix(X)
idx_red_CPU = Matrix(idx_red)
idx_contractile_CPU = Matrix(idx_contractile)
force_CPU = zeros(size((X)))

function runCPU()
    for j = 1:size(idx_red_CPU,1)
        for i =1:size(X,1)
            if idx_red_CPU[j,i] != 0 && idx_red_CPU[j,i] != i
                dist = euclidean(X_CPU,i,idx_red_CPU[j,i])
                force_CPU[i,:] += cubic.(A,r_max,r_min,dist) .* (X_CPU[i,:] - X_CPU[idx_red_CPU[j,i],:]) ./ dist
            end
        end
    end

    for i =1:size(X,1)
        if idx_contractile_CPU[t_knn,i] != i
            dist = euclidean(X_CPU,i,idx_contractile_CPU[t_knn,i])
            force_CPU[i,:] += A_con .* (X_CPU[i,:] - X_CPU[idx_contractile_CPU[t_knn,i],:]) ./ dist
        end
    end
end

runCPU()
display(force_CPU)

1620×3 Matrix{Float64}:
  0.000706573  -0.000870193   0.00482211
  0.000706573  -0.000870193   0.00482211
 -8.91819e-5   -5.35725e-5    0.00573757
 -0.000589626   0.000812196   0.00573757
 -0.000705556  -0.00030851    0.00515901
 -0.000411262  -0.000919341   0.00573757
  0.0           0.000266458   0.00527907
 -0.000499096   0.000550942   0.00446055
 -0.000705556  -0.000142765   0.00470051
  0.000178364   0.00108186    0.00470017
 -0.000251113   0.00170953    0.00540328
  0.0           0.00240127    0.00482271
  0.000705556   0.00117784    0.00482472
  ⋮                          
  0.0          -0.0010096    -0.00458687
 -0.000837609  -0.00128253   -0.00458476
  0.0           0.000311007  -0.00446266
  0.000500444   0.000599311  -0.00527907
 -0.000500444   0.000599311  -0.00527907
  0.00117836   -0.000266458  -0.00527907
 -0.000794738   0.000462795  -0.00515901
 -8.91819e-5    0.000631037  -0.00492116
  0.000705556   0.00030851   -0.00515901
  0.000589626  -0.000812196  -0.00573757
 -0

In [19]:
# testing
@test Matrix(force) ≈ force_CPU atol=0.01

Test Passed